### Prepare the environment

In [26]:
import pandas as pd
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_numeric
import re
from gensim import utils
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamodel
import pyLDAvis.gensim_models as gensim_models

In [27]:
# removed "_" from regular expression
punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^`{|}~"""
RE_PUNCT = re.compile(r'([%s])+' % re.escape(punctuation), re.UNICODE)


def strip_punctuation(s):
    s = utils.to_unicode(s)
    return RE_PUNCT.sub(" ", s)

### Import dataset

In [28]:
df_text = pd.read_excel(open('data/diffusion_innovation_open_ended.xls','rb'), sheet_name='diffusion_innovation_open_ended')

### Knowledge topic model

In [29]:
knowledge_stream = df_text['knowledge'].tolist()
#Preprocess
my_filter = [
    lambda x: x.lower(), strip_tags, strip_punctuation,
    strip_multiple_whitespaces, strip_numeric,
    remove_stopwords, strip_short]

bigram = Phrases(knowledge_stream, min_count=1, threshold=2)
knowledge_bigrams = ["".join(bigram[i]) for i in knowledge_stream]
knowledge_clean = [preprocess_string(i, filters=my_filter) for i in knowledge_bigrams]

print(knowledge_clean[0:5])    

[['share', 'insights'], ['space', 'central', 'person', 'access', 'useful', 'topics', 'trainings', 'accessed'], ['share', 'examples', 'works'], ['continuous', 'exposure', 'upskilling', 'training'], ['spread', 'responsibility', 'great', 'effort', 'upskilling', 'obtaining', 'buy', 'non', 'departments']]


In [30]:
dct_knowledge = Dictionary(knowledge_clean)
corpus = [dct_knowledge.doc2bow(doc) for doc in knowledge_clean]
print(len(dct_knowledge))
print(len(corpus))

435
142


In [31]:
lda = ldamodel.LdaModel(corpus, id2word=dct_knowledge, num_topics = 5)
lda.show_topics(num_words=10)

[(0,
  '0.037*"training" + 0.024*"knowledge" + 0.022*"data" + 0.019*"sharing" + 0.013*"sessions" + 0.013*"online" + 0.013*"exposure" + 0.011*"organization" + 0.009*"systems" + 0.009*"best"'),
 (1,
  '0.038*"people" + 0.027*"training" + 0.016*"knowledge" + 0.012*"technology" + 0.011*"experience" + 0.011*"new" + 0.010*"create" + 0.009*"work" + 0.009*"sessions" + 0.008*"data"'),
 (2,
  '0.098*"training" + 0.025*"sharing" + 0.017*"product" + 0.013*"data" + 0.013*"sessions" + 0.013*"benefits" + 0.013*"workshops" + 0.013*"communication" + 0.011*"knowledge" + 0.009*"tools"'),
 (3,
  '0.047*"training" + 0.034*"data" + 0.022*"employees" + 0.019*"skills" + 0.018*"work" + 0.016*"teams" + 0.013*"online" + 0.010*"forums" + 0.010*"knowledge" + 0.010*"driven"'),
 (4,
  '0.035*"knowledge" + 0.027*"training" + 0.021*"technology" + 0.018*"skills" + 0.016*"company" + 0.016*"tools" + 0.015*"data" + 0.014*"sharing" + 0.011*"sessions" + 0.011*"need"')]

In [32]:
# vis = gensim_models.prepare(lda, corpus, dct_knowledge)
# pyLDAvis.save_html(vis, 'knowledge_lda.html')




import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dct_knowledge)
vis



/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.057293 -0.082622       1        1  27.642757
1      0.099257  0.045870       2        1  21.811506
0     -0.065065  0.038969       3        1  18.522747
2     -0.065343 -0.067811       4        1  18.165781
4     -0.026142  0.065594       5        1  13.857209, topic_info=           Term       Freq      Total Category  logprob  loglift
14     training  39.000000  39.000000  Default  30.0000  30.0000
29       people  10.000000  10.000000  Default  29.0000  29.0000
94   technology   5.000000   5.000000  Default  28.0000  28.0000
122     sharing   9.000000   9.000000  Default  27.0000  27.0000
96    knowledge  14.000000  14.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
35       having   0.708803   2.588930   Topic5  -5.1130   0.6809
116       clear   0.708555   2.487950   Topic5  -5.1134   0.7204
13     exposure   0.708508   3.845847   Topic5  -5.1135   0.2848
119   available   0.708464   2.681355   Topic5  -5.1135   0.6454
220    concepts   0.708011   2.615369   Topic5  -5.1142   0.6697

[299 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
333       1  0.714065       able
2         2  0.764572     access
2         5  0.382286     access
266       5  0.802641   achieves
274       4  0.783840      added
...     ...       ...        ...
34        5  0.132060       work
62        3  0.761501     worked
302       1  0.431312  workshops
302       2  0.215656  workshops
302       4  0.431312  workshops

[334 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3, 5])

### Persuasion topic model

In [33]:
persuasion_stream = df_text['persuasion'].tolist()
print(persuasion_stream[0:10])
#Preprocess
my_filter = [
    lambda x: x.lower(), strip_tags, strip_punctuation,
    strip_multiple_whitespaces, strip_numeric,
    remove_stopwords, strip_short]

#bigram = Phrases(persuasion_stream, min_count=1, threshold=2)
#persuasion_bigrams = ["".join(bigram[i]) for i in persuasion_stream]
persuasion_clean = [preprocess_string(i, filters=my_filter) for i in persuasion_stream]

print(persuasion_clean[0:5])   

['Lead by example, do not lead on opinion', 'Describe the benefits and how it can lead to more informed decisions and the accuracy using a data driven approach rather than gut-feeling or based on misinterpreted analysis.', 'Share examples of how it works', 'Introduction presentations and demos of data driven technologies', 'Selling the value and efficiency that it brings. This might require somebody working closely with the dept to understand were the value add lies.', 'training  workshops', 'Awards/Certificates for participation and different levels of competency. Possible implementation of ideas brought about through the contribution of the participants.', 'Highlight competitive advantage, highlight the existence of trends hidden in data, highlight the criticality of decisions based on trusted information, highlight the need to have an appetite for change (willingness to change direction based on observations within the data).', 'have a prominent display section in for instance the d

TypeError: decoding to str: need a bytes-like object, float found